# Analyze results for 3D images
Jan 21, 2021

In [36]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

from ipywidgets import *

In [37]:
%matplotlib widget

In [38]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_3d_image_analysis import *

# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
# import post_analysis_pandas as post

In [39]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [40]:
img_size=64
dict1={'64':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}


### Read validation data and compute metrics

In [41]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'64':raw_data_dir+'3d_data/dataset1_smoothing_const_params_64cube_100k/val.npy'}

In [42]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_plot_spectrum_3d(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
s_val=f_transform(s_val)
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

# del(s_val)

(1000, 64, 64, 64)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/modules_3d_image_analysis.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


### Read run data

In [43]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [44]:
u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)

In [45]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210210_060657_3d_l0.5_80k', '20210311_220908_3d_l2…

In [46]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210311_220908_3d_l2.0_80k


## Plot Losses

In [47]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [48]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
62491,62491.0,49.0,0.112131,0.077998,0.190129,4.283657,226.869156,222.585495,0.378222,225.409302,0.370602,4.181647,-4.555226,-4.267335,0.190408
62492,62492.0,49.0,0.091194,0.089938,0.181132,4.367055,226.906998,222.539948,0.385244,219.091736,0.392266,4.578687,-4.102508,-4.351802,0.191964
62493,62493.0,49.0,0.063526,0.089113,0.152639,4.028605,223.831177,219.802567,0.390138,225.484558,0.277420,3.899368,-4.294887,-4.006543,0.189273
62494,62494.0,49.0,0.074216,0.079146,0.153363,4.552916,222.508301,217.955383,0.391574,221.558868,0.329833,4.672067,-3.920259,-4.540556,0.189970
62495,62495.0,49.0,0.091820,0.085893,0.177712,4.611991,223.821060,219.209076,0.405940,221.537476,0.364237,4.104641,-4.455357,-4.600916,0.190160
62496,62496.0,49.0,0.086344,0.076625,0.162969,3.961782,217.127487,213.165710,0.399375,210.889450,0.400751,4.036057,-4.288622,-3.939212,0.192068
62497,62497.0,49.0,0.059425,0.085264,0.144689,4.061931,213.600418,209.538483,0.398171,221.424774,0.280305,4.589590,-3.898282,-4.040435,0.190343
62498,62498.0,49.0,0.086120,0.087542,0.173662,4.358519,213.136383,208.777863,0.414464,222.193466,0.273463,4.409338,-4.087823,-4.342693,0.190605
62499,62499.0,49.0,0.074837,0.086497,0.161334,4.595214,209.996490,205.401276,0.413269,209.537811,0.506739,4.522312,-4.057528,-4.581629,0.189788
62500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,221.137634,0.324829,NaN,NaN,NaN,NaN


In [49]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [51]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.332976171374321


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
54872,54872.0,43.0,0.079652,0.077565,0.157217,4.341658,208.962555,204.620895,-1.754332,212.301270,-2.123898,4.382071,-4.399003,-4.324642,0.190272
54919,54919.0,43.0,0.075624,0.084198,0.159822,4.183222,213.552277,209.369049,-1.636441,219.086243,-2.004941,3.964883,-4.178384,-4.164412,0.189841
54889,54889.0,43.0,0.077088,0.101815,0.178903,4.496165,220.830917,216.334747,-1.700051,218.792328,-2.003126,4.153600,-4.788580,-4.480187,0.188564
54885,54885.0,43.0,0.080205,0.068288,0.148494,4.412082,219.114166,214.702087,-1.743845,220.596313,-1.994608,4.150422,-4.255334,-4.387028,0.188304
54828,54828.0,43.0,0.084582,0.113790,0.198371,4.683680,218.629974,213.946289,-1.691278,218.214600,-1.978142,4.217686,-3.918374,-4.671239,0.189627
54869,54869.0,43.0,0.077628,0.082965,0.160592,4.374589,206.724731,202.350143,-1.703115,211.808746,-1.966232,4.002794,-4.365841,-4.357726,0.189059
54811,54811.0,43.0,0.075811,0.078732,0.154543,4.255822,209.334183,205.078354,-1.679540,216.611542,-1.933363,4.229746,-4.276711,-4.238374,0.171903
54788,54788.0,43.0,0.079004,0.080888,0.159891,4.069126,217.729187,213.660065,-1.589655,215.980469,-1.855184,4.218714,-4.534085,-4.049099,0.189996
54829,54829.0,43.0,0.087437,0.094818,0.182255,4.321435,219.135010,214.813568,-1.721216,215.602921,-1.842604,3.736684,-4.751811,-4.304852,0.189781
54824,54824.0,43.0,0.085153,0.065381,0.150534,4.433248,211.225052,206.791809,-1.634127,207.242218,-1.841183,4.814750,-3.935315,-4.418153,0.189512


In [52]:
# display(df_metrics.sort_values(by=['hist_chi']).head(8))
# display(df_metrics.sort_values(by=['spec_chi']).head(8))

## Read stored chi-squares for images

In [53]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [54]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [55]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [56]:
df_sliced=w.result
# df_sliced

In [57]:
best_step=[]
df_test=df.copy()
# df_test=df[df.epoch>5]
best_step.append(f_slice_df(df_test,cutoff=0.9,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df_test,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 73.3058411271215, 'chi_spec1': 289.56761356915416}
[0, 49]
{'chi_1': 50.62032294486212, 'chi_spec1': 194.87936709050285}
[0, 49]
{'chi_1': 6.381113209493947, 'chi_spec1': 39.98681375138315}
[array([56780, 56820, 61520, 61450]), array([52810, 54600]), array([43830, 43920])]
[43830 43920 52810 54600 56780 56820 61450 61520]


In [58]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [59]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(35, 43830), (35, 43920), (42, 52810), (43, 54600), (45, 56780), (45, 56820), (49, 61450), (49, 61520)]


In [60]:
df_best[col_list]


,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
4382,35-43830,5.613139,11.796253,50.416891,5.571642,0.037761,0.003737,1.602585,13.438286,2298.336980,35,43830
4391,35-43920,5.276534,11.885599,50.184576,5.234184,0.038613,0.003737,1.598682,13.471306,2832.150388,35,43920
5280,42-52810,0.140615,17.771909,50.945039,0.087962,0.052351,0.000302,0.045155,7.271102,1284.771954,42,52810
5459,43-54600,0.138005,19.457427,51.354407,0.084841,0.052734,0.000430,0.010106,8.258631,2520.099467,43,54600
5677,45-56780,0.088949,15.843976,47.770792,0.035877,0.050460,0.002611,0.010794,9.841656,1058.310268,45,56780
5681,45-56820,0.097523,15.486176,50.986370,0.041957,0.053795,0.001771,0.018991,8.373916,1127.374221,45,56820
6144,49-61450,0.100988,16.477455,47.701240,0.072862,0.025841,0.002285,0.025154,9.755442,1416.965217,49,61450
6151,49-61520,0.099547,16.189588,50.109877,0.071727,0.025778,0.002042,0.035611,7.947900,1601.990934,49,61520


### Plots 2: stored data

In [61]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    img_size=64
    assert plot_type in ['hist','spec','grid','spec_relative','hist_relative'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative','hist_relative']:     fig=plt.figure(figsize=(6,6))


    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)


        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            
            
        if plot_type=='hist_relative':

            x2=row.hist_bin_centers
            y2=row.hist_val
            yerr2=row.hist_err
            x2=f_invtransform(x2)
            
#             plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['hist_val'],dict_bkg['hist_err']

            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r=sqrt(\delta_c2 ^ 2/ c1^2 + \delta_c1^2 * c_2^2 / c_1^4)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row['num_imgs'])

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)            
            plt.xlim(0,img_size/2)
            plt.ylim(0,5)
            plt.legend()

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:,0]
            print(images.shape)
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.xlim(0,img_size/2)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        plt.xlim(0,img_size/2)
        plt.ylim(0.5,2)
        plt.legend()
        
    if plot_type=='hist_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
        
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [62]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative','hist_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('35-43830', '35-43920', '42-52810', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [ ]:
print(s_val.shape)


In [ ]:
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210123_050147_3d_l0.5_80k/images/gen_img_epoch-28_step-71620.npy'
a1=np.load(fname)

In [ ]:
s_val.shape,a1.shape

In [ ]:
f_plot_grid(a1[6:14,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
f_plot_grid(s_val[100:108,:,:,0],cols=4,fig_size=(4,2))

In [ ]:
df_best[col_list]

In [16]:
dims = [2,3,4,7]
np_x = np.random.uniform(size=dims).astype(np.float32)